In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import signal
from scipy.signal import savgol_filter
import random  
import time
import io
import h5py
import tensorflow as tf
from tensorflow.keras.datasets import mnist     # MNIST dataset is onderdeel van Keras
from tensorflow.keras.models import Sequential  # Het type neuraal netwerk dat we gaan gebruiken
from tensorflow.keras.layers import Dense, Dropout, Activation  # Verschillende type lagen die we gaan gebruiken
from tensorflow.keras.optimizers import SGD, Adam, schedules
from tensorflow.keras import utils
from PIL import Image
import pickle
import sys


configuration = tf.compat.v1.ConfigProto()
configuration.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=configuration)
print("done")

done


# Data prep

In [3]:
def get_pixels_val(image_path): # bron: https://stackoverflow.com/questions/138250/how-to-read-the-rgb-value-of-a-given-pixel-in-python
    """Get a numpy array of an image so that one can access values[x][y]."""
    image = Image.open(image_path, "r")
    pixel_values = np.array(image)

    # width crop
    pixel_values = pixel_values[54:-43]
    
    # height crop
    pv = list()
    for c, h in enumerate(pixel_values):
        pv.append(pixel_values[c, 35:-36])
    
    return np.array(pv)
print("done")

done


# Create transformed and halved csv file

In [12]:
    
folder = r"C:\Users\User\Documents\LUCAS2015_spectra\LUCAS2015_Soil_Spectra_EU28_v2"
def data_clean(countryfile):  
    # opent de og sat data file

    with open(r"C:\Users\User\Documents\LUCAS2015_spectra\LUCAS2015_Soil_Spectra_EU28\spectra_ "+countryfile+" .csv") as f:
        # maakt csv reader aan
        reader = csv.reader(f)
        # Open 
        with open(folder+"\spectra_ "+countryfile+" .csv", 'w', newline='') as file:
            writer = csv.writer(file)
            for c, row in enumerate(reader):
                if c == 0:
                    writer.writerow(row[:5] + row[205:-200:2])
                else:
                    x = np.array(row[205:-200:2], dtype='float64')
                    reflectance = 10**(-x)
                    writer.writerow(row[:5] + list(reflectance))


In [6]:
co_data_path = r"C:\Users\User\Documents\LUCAS2015_topsoildata_20200323\LUCAS_Topsoil_2015_20200323.csv"

col_list = ["Point_ID", "NUTS_0", "OC"]
df = pd.read_csv(co_data_path, usecols=col_list)
max_value = df["OC"].max()
for i in range(20, 561, 20):
    df.loc[(df['OC'] < (i)) & (df['OC'] > (i-20)), ['OC_state']] = f'{i-20}-{i}'
df


,Point_ID,OC,NUTS_0,OC_state
0,34103754,25.5,UK,20-40
1,34443774,503.5,UK,500-520
2,35163814,51.4,UK,40-60
3,32323656,470.3,UK,460-480
4,34463934,43.1,UK,40-60
...,...,...,...,...
21854,45402772,49.9,AT,40-60
21855,47722672,19.4,AT,0-20
21856,45882618,53.9,AT,40-60
21857,45342706,24.2,AT,20-40


In [8]:
countries = ["AT", "BE", "NL", "UK","EL", "EE", "DK", "CZ", "CY","BG", 
             "FI", "FR", "HR", "HU", "IE", "IT", "LT", "LU", 
            "LV", "MT", "PL", "PT", "RO", "SE", "SI", "SK"]
countries = ["BE"]
for country in countries:
    print(country)
    df_country = df.loc[df['NUTS_0'] == country]
    df_country["spectogram"] = np.nan
    pd.options.mode.chained_assignment = None  # turns an annoying warning off, nothing important
    df_country["spectogram"] = df_country["spectogram"].astype(object)

    data_clean(country)
    with open(folder+"\spectra_ " + country +" .csv") as fi:
        re = csv.reader(fi)
        lst = list()
        row1= np.array(next(re)[5:])

        for c, ro in enumerate(re):
            pointid = ro[2]
            if int(pointid) in df_country['Point_ID'].values:
                # make spectogram
                reflectance = np.array(ro[5:])
                r = savgol_filter(reflectance, 11, 2)
                f, t, Sxx = signal.spectrogram(r, 1)
                sxx_sdv = np.array([[ (Sxx[i][j] - np.mean(Sxx[i]) ) / np.std(Sxx[i]) for j in range(Sxx.shape[1])] for i in range(Sxx.shape[0])])
                fig, ax = plt.subplots()
                ax.axis('off')
                ax.pcolormesh(t, f, sxx_sdv, shading='gouraud')#, shading='gouraud'

                # save spectogram in RAM for faster saving
                io_buf = io.BytesIO()
                fig.savefig(io_buf, format='rgba', dpi=72)
                io_buf.seek(0)
                img_arr = np.reshape(np.frombuffer(io_buf.getvalue(), dtype=np.uint8),
                                     newshape=(int(fig.bbox.bounds[3]), int(fig.bbox.bounds[2]), -1))
                io_buf.close()
                plt.close()
                
                # cut white edges off image
                array = img_arr[35:-36]
                array = array[:, 54:-43]
                array = np.delete(array[:], -1, -1)
                # add spectogram do dataframe
                df_country.loc[df_country['Point_ID'] == int(pointid), ['spectogram']] = [array]


    # save dataframe in hdf5 file
    df_country = df_country.dropna()
    df_country.to_hdf(folder+"\labeled_data.hdf5", country)

BE


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:2606: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['NUTS_0', 'OC_state', 'spectogram'], dtype='object')]

  pytables.to_hdf(


57.71775245666504


In [9]:
reread = pd.read_hdf(folder+"\labeled_data.hdf5", key="BE")
reread


,Point_ID,OC,NUTS_0,OC_state,spectogram
3924,39743148,2.3,BE,0-20,"[[[62, 74, 137], [62, 74, 137], [62, 74, 137],..."
3925,39543124,22.4,BE,20-40,"[[[66, 64, 134], [66, 64, 134], [66, 64, 134],..."
3926,39483040,15.2,BE,0-20,"[[[66, 65, 134], [66, 65, 134], [66, 65, 134],..."
3927,39883092,11.9,BE,0-20,"[[[69, 53, 129], [69, 53, 129], [69, 53, 129],..."
3928,39763078,8.6,BE,0-20,"[[[65, 66, 135], [65, 66, 135], [65, 66, 135],..."
...,...,...,...,...,...
21554,39563058,11.4,BE,0-20,"[[[62, 74, 137], [62, 74, 137], [62, 74, 137],..."
21555,39523056,14.3,BE,0-20,"[[[65, 66, 135], [65, 66, 135], [65, 66, 135],..."
21556,39803036,10.0,BE,0-20,"[[[63, 72, 137], [63, 72, 137], [63, 72, 137],..."
21557,40203018,35.8,BE,20-40,"[[[69, 53, 129], [69, 53, 129], [69, 53, 129],..."


In [14]:
"""Converts hdf5 file into a hdf5 file with ptickle 4 protocol for
python 3.7 compatibility"""

pickle.HIGHEST_PROTOCOL = 4
countries = ["AT", "BE", "NL", "UK","EL", "EE", "DK", "CZ", "CY","BG", 
             "FI", "FR", "HR", "HU", "IE", "IT", "LT", "LU", 
            "LV", "MT", "PL", "PT", "RO", "SE", "SI", "SK"]
countries = ["BE"]
for country in countries:
    temppd = pd.read_hdf(folder+"\labeled_data.hdf5", key=country)
    temppd.to_hdf(folder+"\labeled_data_pickle4.hdf5", country)
print('done')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:2606: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['NUTS_0', 'OC_state', 'spectogram'], dtype='object')]

  pytables.to_hdf(


done
